In [1]:
from pprint import pprint
import json

name = 'first-batch';

with open('{}.json'.format(name)) as f:
    data = json.load(f)['logs']
    
with open('images/_airlines.json') as f:
    airlines = json.load(f)

with open('images/_states.json') as f:
    states = json.load(f)

In [3]:
from collections import OrderedDict 
from copy import deepcopy

rows = []

for p in data.items():
    user = p[0]
    d = OrderedDict({
        'id': user
    })
    
    item = p[1]
    
    if not 'done' in item:
        print('ignoring {}'.format(p[0]))
        continue
    
    d['userAgent'] = item['userAgent']
    d['isMobile'] = item['isMobile']
    
    cond = item['cond']
    
    errorAirline = cond['errorAirline']
    seqAirline = cond['seqAirline']
    errorStates = cond['errorStates']
    seqStates = cond['seqStates']
    focusAirline = cond['focusAirline']
    focusState = cond['focusState']
    
    for k, v in cond.items():
        d[k] = v
        
    howManyIndex = 0
    for k, v in item['approximateVis_airline'].items():
        t = v['type']
        
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'approx.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
    
    howManyIndex = 0
    for k, v in item['approximateVis_states'].items():
        t = v['type']
        
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'approx.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
     
    def jaccard_similarity(x,y):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    
    # precise
            
    howManyIndex = 0
    for k, v in item['preciseVis_airline'].items():
        t = v['type']
        
        precise = airlines["{}_{}".format(1, 1)]
        approx = airlines["{}_{}".format(errorAirline, seqAirline)]
        
        prefix = 'precise.airline.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusAirline]
            d[p + 'precise'] = precise[focusAirline]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorAirline > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
            d[p + 'jaccard_approx_precise'] = jaccard_similarity(appr, prec)
            d[p + 'jaccard_approx_answer'] = jaccard_similarity(appr, answers)
            
    howManyIndex = 0
    for k, v in item['preciseVis_states'].items():
        t = v['type']
        
        precise = states["{}_{}".format(1, 1)]
        approx = states["{}_{}".format(errorStates, seqStates)]
        
        prefix = 'precise.states.'
        
        if (t == 'HowMany'):
            p = prefix + 'howMany.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'approx'] = approx[focusState]
            d[p + 'precise'] = precise[focusState]
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'HowManyCompare'):
            p = prefix + 'howManyCompare.' + str(howManyIndex) + '.'
            d[p + 'answer'] = int(v['answer'])
            d[p + 'data'] = v['data']
            d[p + 'approx'] = approx[v['data']]
            d[p + 'precise'] = precise[v['data']]
            d[p + 'confidence'] = v['confidence']
            howManyIndex += 1
            
        if (t == 'DidYouNotice'):
            p = prefix + 'didYouNotice.'
            d[p + 'answer'] = v['answer'] == 'yes'
            d[p + 'true'] = errorStates > 1
            d[p + 'confidence'] = v['confidence']
            
        if (t == 'SelectAll'):
            v = deepcopy(v)
            p = prefix + 'SelectAll.'
            d[p + 'confidence'] = v['confidence']
            
            del v['confidence']
            del v['type']
            
            answers = v.keys()
            prec = [k for k,v in precise.items() if v]
            appr = [k for k,v in approx.items() if v]
            d[p + 'answer'] = ','.join(answers)
            d[p + 'approx'] = ','.join(appr)
            d[p + 'precise'] = ','.join(prec)
            d[p + 'jaccard_answer_precise'] = jaccard_similarity(answers, prec)
    
    for k, v in item['timing'].items():
         d['timing.' + k] = v
            
    for k, v in item['demographics'].items():
         d['demographics.' + k] = v
    
    rows.append(d)

pprint(rows[0])

ignoring 41061051
ignoring 13623650
OrderedDict([('id', '80993444'),
             ('userAgent',
              'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, '
              'like Gecko) Chrome/54.0.2840.71 Safari/537.36'),
             ('isMobile', False),
             ('focusAirline', 'AA'),
             ('errorStates', 1),
             ('focusState', 'NY'),
             ('seqAirline', 4),
             ('errorAirline', 1000),
             ('seqStates', 1),
             ('studyCondition', 'both'),
             ('firstCondition', 'airline'),
             ('secondCondition', 'states'),
             ('approx.airline.howMany.answer', 5000),
             ('approx.airline.howMany.approx', 5000),
             ('approx.airline.howMany.confidence', 'strong_agree'),
             ('approx.airline.howManyCompare.0.answer', 2500),
             ('approx.airline.howManyCompare.0.data', 'EV'),
             ('approx.airline.howManyCompare.0.approx', 2000),
             ('approx.airline

In [5]:
import csv

with open('{}.csv'.format(name), 'w') as f:
    fieldnames = rows[1].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in rows:
        writer.writerow(row)

import pandas as pd

pd.read_csv('{}.csv'.format(name))

,id,userAgent,isMobile,focusAirline,errorStates,focusState,seqAirline,errorAirline,seqStates,studyCondition,...,timing.thanks_html,timing.approximate_first_html,timing.precise_instructions_html,demographics.problems,demographics.troubles,demographics.familiar,demographics.improved,demographics.cheat,demographics.age,demographics.gender
0,80993444,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,AA,1,NY,4,1000,1,both,...,22486,18239,5823,I didn't have any problems.,no,some,No.,no,34,male
1,42878404,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,False,B6,100,NY,8,500,12,difference,...,8623,252895,4809,no issues,no,familiar,-,no,36,male
2,49437914,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4)...,False,DL,1,FL,1,1,1,difference,...,1866427,52356,6908,no,no,familiar,it was all good!,no,22,female
3,23634915,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,AA,1,MA,12,500,1,onlyNew,...,18522,27914,5655,no,no,some,No everything was clear and easy to understand...,no,24,female
4,84835951,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,AA,200,NY,19,1000,14,both,...,60642,21847,7292,-,no,familiar,no,no,27,male
5,55686356,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,False,DL,200,FL,17,200,6,difference,...,2801,20935,2577,no,yes,some,-,no,31,female
6,66972706,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,DL,100,MA,15,500,5,both,...,14204,38587,2711,no,no,familiar,-,no,27,Female
7,58926040,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,False,B6,1000,MA,3,1000,3,onlyNew,...,209269,122610,3239,None at all!,no,familiar,Everything was clear!,no,30,Male
8,55656520,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5...,False,B6,500,MA,20,200,19,onlyNew,...,5809,32904,3250,-,no,some,-,no,33,female
9,18074862,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False,DL,500,NY,1,1,13,both,...,13489,29599,8558,-,no,familiar,-,no,26,male
